In [1]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 100, mode = 'auto')

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

# 학습 및 검증 정확도와 손실 시각화
def plot_training_history(history):
    # 정확도 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # 손실 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

valid_labels = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45]

def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

# hsh 폴더에서 train/test 분리 및 학습 진행
def train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject, test_size=0.2):
    print(f"Training and testing from folder: {subject}")
    
    # hsh 폴더 데이터 로드
    folder = os.path.join(folder_path, subject)
    csv_file = os.path.join(csv_path, f"{subject}.csv")
    image_paths = glob.glob(os.path.join(folder, '*.jpg'))
    csv_data = pd.read_csv(csv_file) if os.path.exists(csv_file) else pd.DataFrame(columns=['Session', 'Point'])
    csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 이미지와 라벨 처리
    data = []
    for img in image_paths:
        session, point = extract_session_and_point(os.path.basename(img))
        if point in valid_labels:
            unique_filename = f"{subject}_{os.path.basename(img)}"
            data.append({
                'Filename': os.path.abspath(img),
                'UniqueFilename': unique_filename,
                'Session': session,
                'Point': point
            })
    df = pd.DataFrame(data)
    merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
    merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
    merged = merged[merged['Point'].isin(valid_labels)]
    
    # Train/Test Split
    train_df, test_df = train_test_split(merged, test_size=test_size, random_state=42, stratify=merged['Point'])
    
    train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
    train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
    
    test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
    test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
    
    # 모델 생성
    right_eye_model = create_resnet18_model((128, 128, 3))
    left_eye_model = create_resnet18_model((128, 128, 3))
    mlp_model = create_mlp_model(train_features.shape[1:])
    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)

    output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = combined_model.fit(
        [train_images_array, train_images_array, train_features], train_labels,
        validation_data=([test_images_array, test_images_array, test_features], test_labels),
        epochs=300,
        batch_size=2,
        verbose=1
    )
    
    # 평가
    predictions = combined_model.predict([test_images_array, test_images_array, test_features])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(test_labels, axis=1)

    loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
    print(f"Testing accuracy for folder {subject}: {accuracy:.4f}")
    
    # 결과 저장
    results = []
    for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
        test_df['Filename'], test_features, predicted_classes, actual_classes)):
        results.append({
            'Subject': subject,
            'Test Accuracy': accuracy,
            'Test Loss': loss,
            'Image File': image_path,
            'Feature': feature.tolist(),
            'Predicted Class': pred_class,
            'Actual Class': actual_class
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(csv_path, f"0107_results_{subject}.csv"), index=False, encoding='utf-8')
    print(f"Results for folder {subject} saved.")
    
    # 모델 저장
    model_save_path = os.path.join(csv_path, f"0107_model_{subject}.h5")
    combined_model.save(model_save_path)
    print(f"Model saved at: {model_save_path}")
    combined_model.summary()
    # 시각화
    plot_training_history(history)

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
subject = 'hsh'

# 동일 폴더에서 train/test 분리 및 실행
train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject)

Training and testing from folder: hsh
Epoch 1/300


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor', 'keras_tensor_65', 'keras_tensor_130']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


460/460 ━━━━━━━━━━━━━━━━━━━━ 108s 199ms/step - accuracy: 0.0480 - loss: 3.9631 - val_accuracy: 0.1000 - val_loss: 2.9984
Epoch 2/300
460/460 ━━━━━━━━━━━━━━━━━━━━ 92s 199ms/step - accuracy: 0.1264 - loss: 2.9006 - val_accuracy: 0.1391 - val_loss: 2.4809
Epoch 3/300
460/460 ━━━━━━━━━━━━━━━━━━━━ 92s 200ms/step - accuracy: 0.2472 - loss: 2.2818 - val_accuracy: 0.4609 - val_loss: 1.5498
Epoch 4/300
460/460 ━━━━━━━━━━━━━━━━━━━━ 92s 200ms/step - accuracy: 0.3334 - loss: 1.8262 - val_accuracy: 0.3391 - val_loss: 2.0217
Epoch 5/300
460/460 ━━━━━━━━━━━━━━━━━━━━ 92s 199ms/step - accuracy: 0.4934 - loss: 1.4806 - val_accuracy: 0.6913 - val_loss: 0.8914
Epoch 6/300
460/460 ━━━━━━━━━━━━━━━━━━━━ 92s 200ms/step - accuracy: 0.6262 - loss: 1.1263 - val_accuracy: 0.8130 - val_loss: 0.6475
Epoch 7/300
460/460 ━━━━━━━━━━━━━━━━━━━━ 92s 200ms/step - accuracy: 0.7100 - loss: 0.9194 - val_accuracy: 0.8435 - val_loss: 0.5370
Epoch 8/300
460/460 ━━━━━━━━━━━━━━━━━━━━ 92s 200ms/step - accuracy: 0.7800 - loss: 0.71


KeyboardInterrupt



In [2]:
import os
import re
import glob
import numpy as np
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

early_stopping = EarlyStopping(monitor = 'val_loss', patience = 100, mode = 'auto')

# Mish activation function
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

# 이미지 로드 및 전처리
def load_and_preprocess_image(image_path, target_size=(128, 128)):
    image = load_img(image_path, color_mode='rgb', target_size=target_size)
    image_array = img_to_array(image)
    image_array /= 255.0
    return image_array

# 파일 이름에서 session과 point 추출
def extract_session_and_point(filename):
    session_match = re.search(r'img_(\d+)', filename)
    point_match = re.search(r'\((\d+)\)', filename)
    session = int(session_match.group(1)) if session_match else None
    point = int(point_match.group(1)) if point_match else None
    return session, point

# ResNet18 모델 생성
def create_resnet18_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    filter_sizes = [64, 128, 256, 512]
    num_blocks = [2, 2, 2, 2]
    for filters, blocks in zip(filter_sizes, num_blocks):
        for i in range(blocks):
            x = resnet_block(x, filters, downsample=(i == 0 and filters != 64))
    x = layers.GlobalAveragePooling2D()(x)
    return Model(inputs=input_layer, outputs=x)

# ResNet 블록 생성
def resnet_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)
    x = layers.Conv2D(filters, kernel_size=(3, 3), strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation(mish)(x)
    x = layers.Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, kernel_size=(1, 1), strides=strides, padding='same')(shortcut)
    x = layers.add([x, shortcut])
    x = layers.Activation(mish)(x)
    return x

# MLP 모델 생성
def create_mlp_model(input_shape):
    input_layer = Input(shape=input_shape)
    x = layers.Dense(128, activation=mish)(input_layer)
    x = layers.Dense(64, activation=mish)(x)
    x = layers.Dense(3, activation=mish)(x)
    x = layers.Flatten()(x)
    return Model(inputs=input_layer, outputs=x)

# 학습 및 검증 정확도와 손실 시각화
def plot_training_history(history):
    # 정확도 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # 손실 시각화
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

valid_labels = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 43, 45]

def prepare_data(folder_path, csv_path, test_subject, valid_labels):
    subject_folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    train_subjects = [s for s in subject_folders if s != test_subject]
    
    # 학습 데이터 로드
    train_images, train_csv_data = [], pd.DataFrame()
    for train_subject in train_subjects:
        train_folder = os.path.join(folder_path, train_subject)
        train_csv = os.path.join(csv_path, f"{train_subject}.csv")
        train_images.extend(glob.glob(os.path.join(train_folder, '*.jpg')))
        if os.path.exists(train_csv):
            train_csv_data = pd.concat([train_csv_data, pd.read_csv(train_csv)])
    train_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 테스트 데이터 로드
    test_folder = os.path.join(folder_path, test_subject)
    test_csv = os.path.join(csv_path, f"{test_subject}.csv")
    test_images = glob.glob(os.path.join(test_folder, '*.jpg'))
    test_csv_data = pd.read_csv(test_csv) if os.path.exists(test_csv) else pd.DataFrame(columns=['Session', 'Point'])
    test_csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 데이터 처리 함수
    def process_images(image_paths, csv_data, valid_labels):
        image_data = []
        for img in image_paths:
            session, point = extract_session_and_point(os.path.basename(img))
            if point in valid_labels:
                subject_name = os.path.basename(os.path.dirname(img))
                unique_filename = f"{subject_name}_{os.path.basename(img)}"
                image_data.append({
                    'Filename': os.path.abspath(img),
                    'UniqueFilename': unique_filename,
                    'Session': session,
                    'Point': point
                })
        df = pd.DataFrame(image_data)
        merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
        merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
        merged = merged[merged['Point'].isin(valid_labels)]
        return merged
    
    train_df = process_images(train_images, train_csv_data, valid_labels)
    test_df = process_images(test_images, test_csv_data, valid_labels)
    return train_df, test_df

# hsh 폴더에서 train/test 분리 및 학습 진행
def train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject, test_size=0.2):
    print(f"Training and testing from folder: {subject}")
    
    # hsh 폴더 데이터 로드
    folder = os.path.join(folder_path, subject)
    csv_file = os.path.join(csv_path, f"{subject}.csv")
    image_paths = glob.glob(os.path.join(folder, '*.jpg'))
    csv_data = pd.read_csv(csv_file) if os.path.exists(csv_file) else pd.DataFrame(columns=['Session', 'Point'])
    csv_data.rename(columns={'session': 'Session', 'point': 'Point'}, inplace=True)
    
    # 이미지와 라벨 처리
    data = []
    for img in image_paths:
        session, point = extract_session_and_point(os.path.basename(img))
        if point in valid_labels:
            unique_filename = f"{subject}_{os.path.basename(img)}"
            data.append({
                'Filename': os.path.abspath(img),
                'UniqueFilename': unique_filename,
                'Session': session,
                'Point': point
            })
    df = pd.DataFrame(data)
    merged = pd.merge(df, csv_data, on=['Session', 'Point'], how='inner')
    merged = merged.drop_duplicates(subset=['UniqueFilename', 'Session', 'Point'])
    merged = merged[merged['Point'].isin(valid_labels)]
    
    # Train/Test Split
    train_df, test_df = train_test_split(merged, test_size=test_size, random_state=42, stratify=merged['Point'])
    
    train_images_array = np.array([load_and_preprocess_image(path) for path in train_df['Filename']])
    train_features = train_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    train_labels = train_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    train_labels = to_categorical(train_labels, num_classes=len(valid_labels))
    
    test_images_array = np.array([load_and_preprocess_image(path) for path in test_df['Filename']])
    test_features = test_df.drop(['Filename', 'UniqueFilename', 'Session', 'Point'], axis=1).values
    test_labels = test_df['Point'].map({label: idx for idx, label in enumerate(valid_labels)}).values
    test_labels = to_categorical(test_labels, num_classes=len(valid_labels))
    
    # 모델 생성
    right_eye_model = create_resnet18_model((128, 128, 3))
    left_eye_model = create_resnet18_model((128, 128, 3))
    mlp_model = create_mlp_model(train_features.shape[1:])
    combined_input = layers.concatenate([right_eye_model.output, left_eye_model.output, mlp_model.output])
    x = layers.Dense(256, activation=mish)(combined_input)
    x = layers.Dropout(0.5)(x)

    output_layer = layers.Dense(len(valid_labels), activation='softmax')(x)
    combined_model = Model(inputs=[right_eye_model.input, left_eye_model.input, mlp_model.input], outputs=output_layer)

    combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    history = combined_model.fit(
        [train_images_array, train_images_array, train_features], train_labels,
        validation_data=([test_images_array, test_images_array, test_features], test_labels),
        epochs=300,
        batch_size=1,
        verbose=1
    )
    
    # 평가
    predictions = combined_model.predict([test_images_array, test_images_array, test_features])
    predicted_classes = np.argmax(predictions, axis=1)
    actual_classes = np.argmax(test_labels, axis=1)

    loss, accuracy = combined_model.evaluate([test_images_array, test_images_array, test_features], test_labels)
    print(f"Testing accuracy for folder {subject}: {accuracy:.4f}")
    
    # 결과 저장
    results = []
    for idx, (image_path, feature, pred_class, actual_class) in enumerate(zip(
        test_df['Filename'], test_features, predicted_classes, actual_classes)):
        results.append({
            'Subject': subject,
            'Test Accuracy': accuracy,
            'Test Loss': loss,
            'Image File': image_path,
            'Feature': feature.tolist(),
            'Predicted Class': pred_class,
            'Actual Class': actual_class
        })
    
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(csv_path, f"0107_results_{subject}.csv"), index=False, encoding='utf-8')
    print(f"Results for folder {subject} saved.")
    
    # 모델 저장
    model_save_path = os.path.join(csv_path, f"0107_model_{subject}.h5")
    combined_model.save(model_save_path)
    print(f"Model saved at: {model_save_path}")
    combined_model.summary()
    # 시각화
    plot_training_history(history)

# 데이터 경로
folder_path = r"C:\Users\admin\Desktop\sihoon\webcam\img"
csv_path = r"C:\Users\admin\Desktop\sihoon\webcam\results"
subject = 'hsh'

# 동일 폴더에서 train/test 분리 및 실행
train_and_evaluate_from_single_folder(folder_path, csv_path, valid_labels, subject)

Training and testing from folder: hsh
Epoch 1/300


C:\Users\admin\.conda\envs\sihoon\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_139', 'keras_tensor_204', 'keras_tensor_269']. Received: the structure of inputs=('*', '*', '*')
  warnings.warn(


920/920 ━━━━━━━━━━━━━━━━━━━━ 159s 156ms/step - accuracy: 0.0549 - loss: 3.8507 - val_accuracy: 0.1174 - val_loss: 3.9626
Epoch 2/300
920/920 ━━━━━━━━━━━━━━━━━━━━ 144s 156ms/step - accuracy: 0.2719 - loss: 2.2017 - val_accuracy: 0.2565 - val_loss: 3.3876
Epoch 3/300
920/920 ━━━━━━━━━━━━━━━━━━━━ 144s 157ms/step - accuracy: 0.5125 - loss: 1.3919 - val_accuracy: 0.5087 - val_loss: 1.3376
Epoch 4/300
920/920 ━━━━━━━━━━━━━━━━━━━━ 143s 156ms/step - accuracy: 0.7100 - loss: 0.8390 - val_accuracy: 0.7261 - val_loss: 0.8590
Epoch 5/300
920/920 ━━━━━━━━━━━━━━━━━━━━ 143s 156ms/step - accuracy: 0.8034 - loss: 0.5565 - val_accuracy: 0.7739 - val_loss: 0.6085
Epoch 6/300
920/920 ━━━━━━━━━━━━━━━━━━━━ 143s 156ms/step - accuracy: 0.8185 - loss: 0.5476 - val_accuracy: 0.5696 - val_loss: 1.4064
Epoch 7/300
920/920 ━━━━━━━━━━━━━━━━━━━━ 143s 156ms/step - accuracy: 0.8633 - loss: 0.3527 - val_accuracy: 0.7087 - val_loss: 0.8728
Epoch 8/300
920/920 ━━━━━━━━━━━━━━━━━━━━ 143s 155ms/step - accuracy: 0.9006 - los

KeyboardInterrupt: 